Aim to complete as much of this tutorial on your own *before* coming to the practical session.

Use the practical session to get help for any aspect you do not understand or were unable to complete.

# Classification and Regression 4

Learning objectives
1. Apply single- and multilayer FNNs in a binary classification setting using the popular python library [sklearn](https://scikit-learn.org/stable/)
2. Visualise the important variables of the single-layer FNN 
3. Explore different metrics to evaluate the model performance in regression settings
4. Explore the effect of changing the network architecture, model parameters and scaling on the model performance and training times

Optional learning objective (view the video online about AEs before you start)

5. Apply AE (autoencoder) for data reduction and using the output for a classification model 

## Import specific packages and functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as mx
import matplotlib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

## Load in dataset

In this tutorial we will use some plasma metabolomics data to predict type-II diabetes occurrence.

In [ ]:
df = pd.read_csv('../Data-main/diabetes_metabolomics_plasma.csv' ) #  can change this to the previous xlsx format  below 
#  pd.read_excel('../Data/diabetes_metabolomics_plasma.xlsx'), add or remove the index_col argument if necissary 

### Inspect the data

In [ ]:
df.head() ### 

In [ ]:
print(df['T2D'].value_counts())
df['T2D'].value_counts().plot(kind='bar')

In python column indices start from 0, we want to subset only the metabolomics colulmns for feature scaling. 
As you see the data is fairly balanced, something to think about is how do our metrics change when there is a class imbalance.

In [ ]:
# Create feature matrix and target vector
X = df.iloc[:,6:]
y = df['T2D']

In order to test our alogrithms we need to set aside some of the data we have. This is practice for machine learning models. We will use 80% of our data to train our model, and the remaining 20% will be used to test the performance of our model. 

Scikit-Learn has a function ```train_test_split``` to easily do this for us.

In [ ]:
# enter your CID here, or date of birth, or another number of your choosing to use as random state
CID = 0

# remember to check the documentation of each algorithm if setting the random_state is needed
# for this tutorial and all upcoming tutorials...

In [ ]:
# Split the df into 80% train 20% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=CID)

It is crucial that all of the data it is comparing is on the same scale. In our proteomics data, most of the data is continuous. We will scale the data using the [StandardScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) shown in the previous tutorials. 

When scaling your data you want to fit the model to your training data, and only transform your testing data. 

In [ ]:
# Instantiate scaler model
scaler = StandardScaler()

# Fit and transform X_train
X_train_scaled = scaler.fit_transform(X_train)

# Transform X_test
X_test_scaled = scaler.transform(X_test)

### Baseline Accuracy

When evaluating model performance we want to start with a baseline accuracy. This is the accuracy score if we were to simply guess the majority outcome everytime. It gives us a starting point to compare our models to. The baseline metric is the best we can do without models. Hopefully, our models can improve over the baseline.

In [ ]:
# Calculate the baseline accuracy

# Find the majority count
y_train.value_counts()# Calculate the baseline accuracy

In [ ]:
#  Counts for y_test
y_test.value_counts()

In [ ]:
# If we were to guess the majority (0) for each test, we would get 42 correct
# baseline is correct guesses divided by total guesses 
baseline = 38 / (38 + 34)

In [ ]:
38 / (38 + 34)

### Type-2 diabetes classification using  MLPClassifier from Sklearn 

We will use the [MLPClassifier()](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) function to create single hidden-layer neural network.

In [ ]:
# Importing MLPClassifier
from sklearn.neural_network import MLPClassifier 

In [ ]:
# Initializing the MLPClassifier (this one is just a single layer of 100 neurons)
classifier = MLPClassifier(hidden_layer_sizes=(100), max_iter=300, activation='relu', solver='adam', random_state=CID)

In [ ]:
# Fitting the training data to the network
classifier.fit(X_train_scaled, y_train)

# Using the trained network to predict y for the test set
y_pred = classifier.predict(X_test_scaled)

Investigate how the coeficents at each layer change, have a think about why the shape changes?

In [ ]:
classifier.coefs_[1].shape  ### Change here the number from try 1 and 0 and look at the shape difference 

In [ ]:
plt.figure(figsize=(20,5))
plt.imshow(classifier.coefs_[0][0:20], interpolation='none', cmap='viridis')
plt.yticks(range(20), np.array(X.columns)[0:20])
plt.xlabel("Columns in weight matrix")
plt.ylabel("Input feature")
plt.colorbar()

## Interpretation of Figure

- Shows weights learned connecting the input to the first hidden layer
- Rows in the plot correspond to the 20 input features
- Columns in plot correspond to the 100 hidden units


- Feature that have very small weights for all hidden units are 'less important' to model
- Could also visualize weight connecting the hidden layers to the output layer, but that is even harder to interpret

### Metrics 


In [ ]:
## Accuracy is one performance metric here we will define several alternative metrics 

def modelPerformance(confMat):
    TN = confMat[0, 0]
    TP = confMat[1, 1]
    FP = confMat[0, 1]
    FN = confMat[1, 0]
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    spec = TN / (TN + FP)
    fpr = FP / (TN + FP)
    f1 = 2 * (prec * rec) / (prec + rec)
    acc = (TP + TN) / (TP + FP + TN + FN)
    return (acc, prec, rec, spec, fpr, f1)

def printPerformance(confMat):
    acc, prec, rec, spec, fpr, f1 = modelPerformance(confMat)
    print("Accuracy = " "%.4f" % acc)
    print("Precision = " "%.4f" % prec)
    print("Recall = " "%.4f" % rec)
    print("Specificity = " "%.4f" % spec)
    print("False positive rate = " "%.4f" % fpr)
    print("F1-score = " "%.4f" % f1)
    np.set_printoptions(precision=2)
    print("Confusion matrix (%):")
    print(confMat/np.sum(confMat)*100)

In [ ]:
cmat = confusion_matrix(y_test, y_pred)
printPerformance(cmat)

Here we are achieving an accuracy of 0.82, lets see if we can improve upon this with more layers:

In [ ]:
# Initializing the MLPClassifier (this one has layers of 100 and 200 neurons)
classifier = MLPClassifier(hidden_layer_sizes=(100,200), max_iter=300, activation='relu', solver='adam', random_state=CID)

# Fitting the training data to the network
classifier.fit(X_train_scaled, y_train)

# Using the trained network to predict y for the test set
y_pred = classifier.predict(X_test_scaled)

In [ ]:
# Confusion matrix and metrics
cmat = confusion_matrix(y_test, y_pred)
printPerformance(cmat)

Here we are achieving an accuracy of 0.83, lets see if we can improve upon this with 10-fold CV and some hyperparameter tuning. We can use the [GridSearchCV()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) function to help with this. Remember you can change `GridSearchCV` below for another method...

In [ ]:
parameters = {'solver': ['adam'], 'max_iter': [300,500,700], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':[(50,100),(100,200),(50, 100,200), (100,200,300)] , 'random_state':[CID], 'activation':['relu']}
clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)

In [ ]:
parameters

In [ ]:
### this may take 2-5 minutes to run depending on your hardware
clf.fit(X_train_scaled, y_train)

In [ ]:
print(clf.best_params_)

In [ ]:
#Predicting y for X_val
y_pred = clf.predict(X_test_scaled)

In [ ]:
cmat = confusion_matrix(y_test, y_pred)
printPerformance(cmat)

Great we can achieve a reasonable test accuracy. Now try in the cell below some of your own hyperparameters in the gridsearch setting and see if you can achieve a higher performance?
This [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) will help you choose different options.

In [ ]:
### Try it yourself here with parameters of your choice 
parameters = {'solver': ['adam'], 'max_iter': [123,456,789], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':[(1,2),(3,4)] , 'random_state':[42], 'activation':['relu']}
clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)

### this may take 2-5 minutes to run depending on your hardware and no. hyperperamters
clf.fit(X_train_scaled, y_train)

# Using the trained network to predict y for the test set
y_pred = clf.predict(X_test_scaled)

cmat = confusion_matrix(y_test, y_pred)
printPerformance(cmat)

# Optional materials
Please view the [Youtube video](https://youtu.be/k8TID4tvew8) online about AEs before you start this part of the tutorial.

Now that you have understood how to implement FNNs, let's aim to implement another common neural network: an autoencoder. 

## Autoencoders (AEs)

We will use the [MLPRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) function for this.

In [ ]:
# We will make an simple autoencoder from the MLPRegressor function in sklearn
from sklearn.neural_network import MLPRegressor

In [ ]:
# Shape of latent variable and hidden layers

# Encoder structure
n_encoder1 = 300
n_encoder2 = 500

n_latent = 50

# Decoder structure
n_decoder2 = 300
n_decoder1 = 500

Have a look at the documentation whether or not you need to set a `random_state` or not.

In [ ]:
# We will make an simple autoencoder from the MLPRegressor function in sklearn
reg = MLPRegressor(hidden_layer_sizes = (n_encoder1, n_encoder2, n_latent, n_decoder2, n_decoder1), 
                   activation = 'tanh', 
                   solver = 'adam', 
                   learning_rate_init = 0.001, 
                   max_iter = 100, # This is where we define the number of iterations (or epochs) in sklearn
                   tol = 0.000001, 
                   verbose = False)

In [ ]:
# fit the model with trainin data  as input an dX as output 
# This may take 5 minutes depending on your hardware, if training takes too long decrease max_iter in the above cell
# It may not converge but for this tutorial that will be fine 
reg.fit(X_train_scaled, X_train_scaled)

In [ ]:
# Encoder helper function, no need to change this 

def encoder(data):
    data = np.asmatrix(data)
    
    encoder1 = data*reg.coefs_[0] + reg.intercepts_[0]
    encoder1 = (np.exp(encoder1) - np.exp(-encoder1))/(np.exp(encoder1) + np.exp(-encoder1))
    
    encoder2 = encoder1*reg.coefs_[1] + reg.intercepts_[1]
    encoder2 = (np.exp(encoder2) - np.exp(-encoder2))/(np.exp(encoder2) + np.exp(-encoder2))
    
    latent = encoder2*reg.coefs_[2] + reg.intercepts_[2]
    latent = (np.exp(latent) - np.exp(-latent))/(np.exp(latent) + np.exp(-latent))
    
    return np.asarray(latent)



## Examine the Latent Space of the Autoencoder

In [ ]:
# Extract latent embeddings with trained encoder part of autoencoder
test_latent = encoder(X_test_scaled)
train_latent = encoder(X_train_scaled)

In [ ]:
plt.scatter(train_latent[:,0], train_latent[:,1], c=y_train, s=50, cmap='viridis')
plt.show()

Like in previous tutorials we can also examine this latent space with PCA compnents

In [ ]:
from sklearn.decomposition import PCA, SparsePCA, KernelPCA
pca = PCA(n_components=4)
pca_latent = pca.fit_transform(train_latent)
# run PCA with 4 components
# plot a scatterplot using seaborn
# the x axis will contain the first column of the pca scores x=pca_covid[:, 0]
plt.scatter(pca_latent[:, 0], pca_latent[:, 1], c=y_train, s=50, cmap='viridis')
plt.show()

You can also use _any_ of the methods from BIDS 2-4 (e.g. [SparsePCA()](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html), [Kernel PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html#sklearn.decomposition.KernelPCA), [umap.UMAP()](https://umap-learn.readthedocs.io/en/latest/index.html), etc).

### We will now train an FFNN on this autoencoder generated latent space to predict T2 Diabetes

In [ ]:
parameters = {'solver': ['adam'], 'max_iter': [300,500,700], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':[(50,100),(100,200),(50, 100,200), (100,200,300)] , 'random_state':[CID], 'activation':['relu']}
clf_lat = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)

In [ ]:
### this may take 2-5 minutes to run depending on your hardware
clf_lat.fit(train_latent, y_train)

In [ ]:
# Predicting y for X_val
y_pred = clf_lat.predict(test_latent)
cmat = confusion_matrix(y_test, y_pred)
printPerformance(cmat)

We can see that it is not performing as well as some of our previous models, however we have not tuned the hyperparamaters of this model. This can take a few hours to train an autoencoder to get a good performance. In an ideal setting we would itterate over various paramters however for today you can try change the dimensions of the latent space and see if you can get an improvement. 

### We can also use PCA (or other methods) for dimensionality reduction and build models on the PCA components

In [ ]:
pca = PCA(n_components=50) # if you've changed the latent space dimension for AEs, change it here too
train_components = pca.fit_transform(X_train_scaled)
test_components = pca.transform(X_test_scaled)

Remember you can change `GridSearchCV` below for another method...

In [ ]:
parameters = {'solver': ['adam'], 'max_iter': [300,500,700], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':[(50,100),(100,200),(50, 100,200), (100,200,300)] , 'random_state':[CID], 'activation':['relu']}
clf_pca= GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)

### this may take 2-5 minutes to run depending on your hardware
clf_pca.fit(train_components, y_train)

#Predicting y for test data 
y_pred = clf_pca.predict(test_components)
cmat = confusion_matrix(y_test, y_pred)
printPerformance(cmat)

Although the autoencoder did not perform the best (depsite being a neural network), perhaps with further hyperparameter tuning and training it could.

Considering the baseline test accuracy was 54%, our neural networks have shown an improvement with an accuracy of 84%.

# What's Next?
We have walked through how to implement MLPClassifiers, MLPRegression (autoencoder) 
- Its important to note the MLPClassifier and MLPRegression only capture a small amount of what is possible with deep learning.
- Deep learning involves many of the concepts we have covered in BIDS, from regularisation to kernels, however it also typically requires larger datasets to extract the relevant data structures.
- If you want to explore deep learning after completing BIDS, you can start with different frameworks such as [TensorFlow](https://www.tensorflow.org), [PyTorch](https://pytorch.org) and [keras](https://keras.io).

For now, try to further your understanding and practice with simpler NNs:
- Try using a different scaling for instance: ```robust_scale``` / ```RobustScaler```, ```power_transform``` / ```PowerTransformer``` as these may considerably change your model performance.
- Change certain parameters such as latent space dimensions (number of components) in PCA.
- Use a different dataset for a regression problem.
- Try run a GridSearchCV on the Autoencoder model (note this will increase training time and RandomizedSearchCV will be faster!)